# Check if you're using GPU to accelerate your trainings

In [ ]:
!nvidia-smi

# The following cells will clone darknet from AlexeyAB's famous repository

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

# Adjust the Makefile to enable OPENCV and GPU for darknet and then build darknet

In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

# Verify CUDA version
****

In [ ]:
!/usr/local/cuda/bin/nvcc --version

# Builds darknet so that we can use the darknet executable file to run or train object detectors

In [ ]:
!make

# YOLOv4 has been trained already on the coco dataset which has 80 classes that it can predict. We will grab these pretrained weights so that we can run YOLOv4 on these pretrained classes and get detections.

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

# These three functions are helper functions that will allow you to show the image in your Cloud VM after running your detections, as well as upload and download images to and from it

In [ ]:
# define helper functions
def imShow(path):
    import cv2
    import matplotlib.pyplot as plt
    %matplotlib inline

    image = cv2.imread(path)
    height, width = image.shape[:2]
    resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

    fig = plt.gcf()
    fig.set_size_inches(18, 10)
    plt.axis("off")
    plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
    plt.show()

# use this to upload files
def upload():
    from google.colab import files
    uploaded = files.upload() 
    for name, data in uploaded.items():
    with open(name, 'wb') as f:
        f.write(data)
        print ('saved file', name)

# use this to download a file  
def download(path):
    from google.colab import files
    files.download(path)

# Mount your drive on your cloud VM

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

# Creation of symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive

In [ ]:

!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

# Copy over both datasets into the root directory of the Colab VM

In [ ]:
!cp /mydrive/yolov4/obj.zip ../
!cp /mydrive/yolov4/test.zip ../

# Unzip the datasets and their contents so that they are now in /darknet/data/ folder

In [ ]:
!unzip ../obj.zip -d data/
!unzip ../test.zip -d data/

# This step involves properly configuring your custom .cfg, obj.data, obj.names, train.txt and test.txt files.

# It is important to configure all these files with extreme caution as typos or small errors can cause major problems with your custom training.

I recommend having batch = 64 and subdivisions = 16 for ultimate results. If you run into any issues then up subdivisions to 32.

Make the rest of the changes to the cfg based on how many classes you are training your detector on.

Note: I set my max_batches = 6000, steps = 4800, 5400, I changed the classes = 1 in the three YOLO layers and filters = 18 in the three convolutional layers before the YOLO layers.

How to Configure Your Variables:

width = 416

height = 416 (these can be any multiple of 32, 416 is standard, you can sometimes improve results by making value larger like 608 but will slow down training)

max_batches = (# of classes) * 2000 (but no less than 6000 so if you are training for 1, 2, or 3 classes it will be 6000, however detector for 5 classes would have max_batches=10000)

steps = (80% of max_batches), (90% of max_batches) (so if your max_batches = 10000, then steps = 8000, 9000)

filters = (# of classes + 5) * 3 (so if you are training for one class then your filters = 18, but if you are training for 4 classes then your filters = 27)

Optional: If you run into memory issues or find the training taking a super long time. In each of the three yolo layers in the cfg, change one line from random = 1 to random = 0 to speed up training but slightly reduce accuracy of model. Will also help save memory if you run into any memory issues.

Copy configuration file to root after change it

In [ ]:
!cp /mydrive/yolov4/yolov4-obj.cfg ./cfg

Create a new file within a code or text editor called obj.names where you will have one class name per line in the same order as your classes.txt from the dataset generation step.

In [ ]:
!cp /mydrive/yolov4/obj.names ./data

# You will also create a obj.data file and fill it with 
1. Number of classes
2. Path to train.txt
3. Path to test.txt
4. Path to Classes.txt
5. Path to Backup ; This backup path is where we will save the weights to of our model throughout training

In [ ]:
!cp /mydrive/yolov4/obj.data  ./data

The last configuration files needed before we can begin to train our custom detector are the train.txt and test.txt files which hold the relative paths to all our training images and valdidation images.

Luckily I have created scripts that eaily generate these two files withe proper paths to all images.

The scripts can be accessed from the Github Repo

Just download the two files to your local machine and upload them to your Google Drive so we can use them in the Colab Notebook.

In [ ]:
!cp /mydrive/yolov4/generate_train.py ./
!cp /mydrive/yolov4/generate_test.py ./

Now simply run both scripts to do the work for you of generating the two txt files.

In [ ]:
!python generate_train.py
!python generate_test.py

This step downloads the weights for the convolutional layers of the YOLOv4 network. By using these weights it helps our custom object detector to be way more accurate and not have to train as long. We don't have to use these weights but it will help our model converge and be accurate way faster

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

# Train your custom detector

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show -map